In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame , Series
import math

In [3]:
df = pd.read_csv("acc.csv")
df = df[['date','open','high','low','close','volume']]
df=df.loc[::-1].reset_index()
df = df.drop(columns='index')
df

,date,open,high,low,close,volume
0,01-Jan-03,166.00,167.00,164.00,164.35,252826
1,02-Jan-03,164.10,166.25,163.05,164.85,621888
2,03-Jan-03,165.60,166.25,158.20,159.25,1121222
3,06-Jan-03,160.10,160.25,156.50,157.05,553635
4,07-Jan-03,158.55,158.55,152.35,153.85,1169466
...,...,...,...,...,...,...
4626,05-Aug-21,2400.05,2413.70,2375.20,2392.20,399193
4627,06-Aug-21,2382.10,2398.80,2335.40,2341.35,516746
4628,09-Aug-21,2348.00,2366.55,2304.00,2312.60,449768
4629,10-Aug-21,2324.00,2330.00,2266.95,2296.95,321608


In [4]:
def sma(data,period: int = 10,column: str ='close') -> Series:
    return pd.Series(
        data[column].rolling(window = period,min_periods= 1).mean(),
        name = f'{period}_SMA'
    )
sma(df)

0        164.350000
1        164.600000
2        162.816667
3        161.375000
4        159.870000
           ...     
4626    2387.835000
4627    2388.215000
4628    2382.555000
4629    2375.080000
4630    2363.295000
Name: 10_SMA, Length: 4631, dtype: float64

In [ ]:
def smm(data,period: int = 10,column: str ='close') -> Series:
    return pd.Series(
        data[column].rolling(window = period,min_periods= 1).median(),
        name = f'{period}_SMM'
    )
smm(df)

In [3]:
def ssma(data,period: int = 10,column: str ='close',adjust: bool = True) -> Series:
    return pd.Series(
        data[column].ewm(ignore_na = False, alpha=1.0/period, min_periods=0, adjust=adjust).mean(),
        name = f'{period}_SSMA'
    )
ssma(df)

0        164.350000
1        164.613158
2        162.634133
3        161.010366
4        159.261846
           ...     
4626    2293.627514
4627    2298.399762
4628    2299.819786
4629    2299.532808
4630    2297.184527
Name: 10_SSMA, Length: 4631, dtype: float64

In [23]:
def ema(data,period: int = 10,column: str ='close',adjust: bool = True) -> Series:
    return pd.Series(
        data[column].ewm(span=period, adjust=adjust).mean(),
        name = f'{period}_EMA'
    )
ema(df)

0        164.350000
1        164.625000
2        162.464286
3        160.680520
4        158.719663
           ...     
4626    2364.702918
4627    2360.456933
4628    2351.755672
4629    2341.791005
4630    2329.838095
Name: 10_EMA, Length: 4631, dtype: float64

In [28]:
def dema(data,period: int = 10,column: str ='close',adjust: bool = True) -> Series:
    DEMA = (
    2*ema(data,period) - ema(data,period).ewm(span=period , adjust=adjust).mean()
    )
    return pd.Series(
        DEMA ,
        name = f'{period}_DEMA'
    )
dema(df)

0        164.350000
1        164.748750
2        166.224675
3        166.697002
4        166.701150
           ...     
4626    2403.913810
4627    2437.237834
4628    2458.550351
4629    2469.641106
4630    2471.518337
Name: 10_DEMA, Length: 4631, dtype: float64

In [ ]:
def tema(data,period: int = 10,column: str ='close',adjust: bool = True) -> Series:
    triple_ema = 3 * ema(data,period)
    ema_ema_ema = (
        ema(data,period).ewm(ignore_na = False, span = period, adjust = adjust).mean()
        .ewm(ignore_na = False, span = period, adjust = adjust).mean()
    )
    TEMA = (
    triple_ema - 3 * ema(data,period).ewm(span=period, adjust= adjust).mean() + ema_ema_ema
    )
    return pd.Series(
        TEMA ,
        name = f'{period}_TEMA'
    )
tema(df)

In [ ]:
def trima(data,period: int = 10,adjust: bool = True) -> Series:
    SMA = sma(data,period).rolling(window=period , min_periods=1).sum()
    return pd.Series(
        SMA / period,
        name = f'{period}_TRIMA'
    )
trima(df)

In [ ]:
def trix(data,period: int = 10,adjust: bool = True,column: str ='close') -> Series:
    data_ = data[column]
    def _ema(data_, period, adjust):
        return pd.Series(data_.ewm(span=period, adjust=adjust).mean())

    m = _ema(_ema(_ema(data_, period, adjust), period, adjust), period, adjust)
    return pd.Series(
        10000 * (m.diff() / m), 
        name = f'{period}_TRIX'
        )
trix(df)

In [24]:
def vama(data,period: int = 10,column: str ='close') -> Series:
    vp = data[column]*data['volume']
    volsum = data["volume"].rolling(window=period,min_periods=1).mean()
    volRatio = pd.Series(vp / volsum, name="VAMA")
    cumSum = (volRatio * data[column]).rolling(window=period,min_periods=1).sum()
    cumDiv = volRatio.rolling(window=period,min_periods=1).sum()
    
    return pd.Series(cumSum / cumDiv, name=f'{period}_VAMA')

vama(df)

0        164.350000
1        164.643921
2        162.474024
3        161.553222
4        159.770848
           ...     
4625    2364.579425
4626    2378.634755
4627    2381.035508
4628    2371.219487
4629    2361.572551
Name: 10_VAMA, Length: 4630, dtype: float64

In [ ]:
def er(data,period: int = 10,column: str ='close') -> Series:
    change = data[column].diff(period).abs()
    volatility = data[column].diff().abs().rolling(window=period,min_periods=1).sum()

    return pd.Series(change / volatility, name=f'{period}_ER')

    
    

er(df)

In [ ]:
def kama(data,er_: int = 10,ema_fast: int = 2,
         ema_slow: int = 30,period: int = 20,
         column: str ='close') -> Series:
    er_ = er(data)
    fast_alpha = 2 / (ema_fast + 1)
    slow_alpha = 2 / (ema_slow + 1)
    sc = pd.Series(
            (er_ * (fast_alpha - slow_alpha) + slow_alpha) ** 2,
            name="smoothing_constant",
        )
    sma = pd.Series(
            data[column].rolling(period).mean(), name="SMA"
        )
    kama = []
    for s, ma, price in zip(
            sc.iteritems(), sma.shift().iteritems(), data[column].iteritems()
        ):
            try:
                kama.append(kama[-1] + s[1] * (price[1] - kama[-1]))
            except (IndexError, TypeError):
                if pd.notnull(ma[1]):
                    kama.append(ma[1] + s[1] * (price[1] - ma[1]))
                else:
                    kama.append(None)
    sma["KAMA"] = pd.Series(
            kama, index=sma.index,  name=f'{period}_KAMA')

    return sma['KAMA']

    
    

kama(df)

In [ ]:
def zlema(data,period: int = 26, adjust: bool = True,
       column: str = "close") -> Series:
    lag = (period - 1) / 2

    ema = pd.Series(
            (data[column] + (data[column].diff(lag))),
            name=f'{period}_ZLEMA')
    zlema = pd.Series(
            ema.ewm(span=period, adjust=adjust).mean(),
            name=f'{period}_ZLEMA'
        )
    return zlema
zlema(df)

In [ ]:
def wma(data, period: int = 9, 
        column: str = "close") -> Series:
    d = (period * (period + 1))/2
    weights = np.arange(1, period + 1)
    
    def linear(w):
            def _compute(x):
                return (w * x).sum() / d

            return _compute

    _close = data[column].rolling(period, min_periods=period)
    wma = _close.apply(linear(weights), raw=True)

    return pd.Series(wma, name=f'{period}_WMA')

wma(df)

In [ ]:
def hma(data, period: int = 16) -> Series:
    
    half_length = int(period / 2)
    sqrt_length = int(math.sqrt(period))

    wmaf = wma(data, period=half_length)
    wmas = wma(data, period=period)
    data["deltawma"] = 2 * wmaf - wmas
    hma = wma(data, column="deltawma", period=sqrt_length)

    return pd.Series(hma, name=f'{period}_HMA')

hma(df)


In [9]:
def evwma(data, period: int = 20) -> Series:
    vol_sum = (data["volume"].rolling(window=period,min_periods=1).sum())

    x = (vol_sum - data["volume"]) / vol_sum
    y = (data["volume"] * data["close"]) / vol_sum
    
    evwma = [0]
    
    for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
            if x[1] == 0 or y[1] == 0:
                evwma.append(0)
            else:
                evwma.append(evwma[-1] * x[1] + y[1])

    return pd.Series(
        evwma[1:], index=data.index, name=f'{period}_EVWMA')
evwma(df)

0          0.000000
1        117.202008
2        145.176153
3        149.876946
4        155.076158
           ...     
4626    2087.986786
4627    2097.285104
4628    2104.964020
4629    2110.315139
4630    2116.616747
Name: 20_EVWMA, Length: 4631, dtype: float64

In [ ]:
def tp(data: DataFrame) -> Series:
        return pd.Series((data["high"] + data["low"] + data["close"]) / 3, name="TP")
tp(df)

In [ ]:
def mvwap(data: DataFrame, period:int = 9) -> Series:
        data["cv"] =(data["close"] * data["volume"])
        return pd.Series(
            (sma(data,period = period,column = "cv")/sma(data,period=period,column="volume")),
            name="MVWAP."
        )
mvwap(df)

In [ ]:
def vwap(data: DataFrame) -> Series:
        return pd.Series(
            ((data["volume"] * tp(data)).cumsum()) / data["volume"].cumsum(),
            name="VWAP.",
        )
vwap(df)

In [ ]:
def smma(data, period: int = 42,column: str = "close",
        adjust: bool = True) -> Series:
    return pd.Series(
            data[column].ewm(alpha=1 / period, adjust=adjust).mean(), name=f'{period}_SMMA'
        )
smma(df)

In [ ]:
def FRAMA(data: DataFrame, period: int = 16, batch: int=10) -> Series:

        assert period % 2 == 0, print("FRAMA period must be even")

        c = data.close.copy()
        window = batch * 2

        hh = c.rolling(batch).max()
        ll = c.rolling(batch).min()

        n1 = (hh - ll) / batch
        n2 = n1.shift(batch)

        hh2 = c.rolling(window).max()
        ll2 = c.rolling(window).min()
        n3 = (hh2 - ll2) / window

        # calculate fractal dimension
        D = (np.log(n1 + n2) - np.log(n3)) / np.log(2)
        alp = np.exp(-4.6 * (D - 1))
        alp = np.clip(alp, .01, 1).values

        filt = c.values
        for i, x in enumerate(alp):
            cl = c.values[i]
            if i < window:
                continue
            filt[i] = cl * x + (1 - x) * filt[i - 1]

        return pd.Series(filt, index=data.index, name="{0} period FRAMA.".format(period))


In [ ]:
def macd(data,period_fast: int = 12,period_slow: int = 26,
        signal: int = 9,column: str = "close",adjust: bool = True
    ) -> DataFrame:
    
    EMA_fast = pd.Series(
            data[column].ewm(ignore_na=False, span=period_fast, adjust=adjust).mean(),
            name=f'{period_fast}_EMA_fast')
    EMA_slow = pd.Series(
        data[column].ewm(ignore_na=False, span=period_slow, adjust=adjust).mean(),
        name=f'{period_slow}_EMA_slow')
    MACD = pd.Series(EMA_fast - EMA_slow,name='MACD')
    MACD_signal = pd.Series(
        MACD.ewm(ignore_na=False, span=signal, adjust=adjust).mean(),name=f'{signal}_SIGNAL'
    )
    DIFF = pd.Series(
        MACD - MACD_signal,
        name="diff MACD_MSIGNAL"
    )

    return pd.concat([DIFF, MACD, MACD_signal ], axis=1)

macd(df)

In [ ]:
def pivot_traditional(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot = pd.Series(tp(df_), name="pivot")
    
    s1 = (pivot * 2) - df_["high"]
    s2 = pivot - (df_["high"] - df_["low"])
    s3 = df_["low"] - (2 * (df_["high"] - pivot))
    s4 = df_["low"] - (3 * (df_["high"] - pivot))
    s5 = df_["low"] - (4 * (df_["high"] - pivot))
    

    r1 = (pivot * 2) - df["low"]
    r2 = pivot + (df_["high"] - df_["low"])
    r3 = df_["high"] + (2 * (pivot - df_["low"]))
    r4 = df_["high"] + (3 * (pivot - df_["low"]))
    r5 = df_["high"] + (4 * (pivot - df_["low"]))
    return pd.concat(
            [
                pivot,
                pd.Series(s1, name="s1"),
                pd.Series(s2, name="s2"),
                pd.Series(s3, name="s3"),
                pd.Series(s4, name="s4"),
                pd.Series(s5, name="s5"),
                pd.Series(r1, name="r1"),
                pd.Series(r2, name="r2"),
                pd.Series(r3, name="r3"),
                pd.Series(r4, name="r4"),
                pd.Series(r5, name="r5"),
            ],
            axis=1,
        )
pivot_traditional(df)

In [ ]:
def pivot_fibonacci(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot = pd.Series(tp(df_), name="pivot")
    
    s1 = pivot - ((df_["high"] - df_["low"])*0.382)
    s2 = pivot - ((df_["high"] - df_["low"])*0.618)
    s3 = pivot - (df_["high"] - df_["low"])
    s4 = pivot + ((df_["high"] - df_["low"])*1.382)
   
    

    r1 = pivot + ((df_["high"] - df_["low"])*0.382)
    r2 = pivot + ((df_["high"] - df_["low"])*0.618)
    r3 =pivot + (df_["high"] - df_["low"])
    r4 = pivot + (df_["high"] - df_["low"])*1.382
   
    return pd.concat(
            [
                pivot,
                pd.Series(s1, name="s1"),
                pd.Series(s2, name="s2"),
                pd.Series(s3, name="s3"),
                pd.Series(s4, name="s4"),
                pd.Series(r1, name="r1"),
                pd.Series(r2, name="r2"),
                pd.Series(r3, name="r3"),
                pd.Series(r4, name="r4"),
                            ],
            axis=1,
        )
pivot_fibonacci(df)

In [ ]:
def pivot_woodie(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot = pd.Series((df_['high']+df_['low']+2*data['open'])/4, name="pivot")
    
    s1 =  2*pivot-df_['high']
    s2 = pivot - (df_["high"] - df_["low"])
    s3 = df_["low"] - (2 * (pivot - df_["high"]))
    s4 =  s3 - (df_["high"] - df_["low"])
   
    

    r1 = 2*pivot-df_['low']
    r2 = pivot + (df_["high"] - df_["low"])
    r3 =df_["high"] + (2 * (pivot - df_["low"]))
    r4 =  r3 + (df_["high"] - df_["low"])
   
    return pd.concat(
            [
                pivot,
                pd.Series(s1, name="s1"),
                pd.Series(s2, name="s2"),
                pd.Series(s3, name="s3"),
                pd.Series(s4, name="s4"),
                pd.Series(r1, name="r1"),
                pd.Series(r2, name="r2"),
                pd.Series(r3, name="r3"),
                pd.Series(r4, name="r4"),
                            ],
            axis=1,
        )
pivot_woodie(df)

In [ ]:
def pivot_classic(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot = pd.Series(tp(df_), name="pivot")
    
    s1 = (pivot * 2) - df_["high"]
    s2 = pivot - (df_["high"] - df_["low"])
    s3 = pivot - 2*(df_["high"] - df_["low"])
    s4 = pivot - 3*(df_["high"] - df_["low"])
    
    

    r1 = (pivot * 2) - df["low"]
    r2 = pivot + (df_["high"] - df_["low"])
    r3 = pivot + 2*(df_["high"] - df_["low"])
    r4 = pivot + 3*(df_["high"] - df_["low"])
   
    return pd.concat(
            [
                pivot,
                pd.Series(s1, name="s1"),
                pd.Series(s2, name="s2"),
                pd.Series(s3, name="s3"),
                pd.Series(s4, name="s4"),
                pd.Series(r1, name="r1"),
                pd.Series(r2, name="r2"),
                pd.Series(r3, name="r3"),
                pd.Series(r4, name="r4"),
               
            ],
            axis=1,
        )
pivot_classic(df)

In [ ]:
def pivot_demark(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot,s1,r1=[],[],[]
    for i in range(len(df_)):
        if df_['open'][i]==df_['close'][i]:
            x=df_['high'][i]+df_['low'][i]+2*df_['close'][i]
        elif df_['close'][i]>df_['open'][i]:
            x=2*df_['high'][i]+df_['low'][i]+df_['close'][i]
        else:
            x=df_['high'][i]+2*df_['low'][i]+df_['close'][i]
   
        pivot.append(x/4)
        s1.append(x/2 - df_["high"][i])

        r1.append(x/2 - df["low"][i])
    
    data_ = pd.DataFrame(pivot,columns=['pivot'])
    data_['s1']=s1
    data_['r1']=r1
    return data_
        
pivot_demark(df)

In [ ]:
def pivot_camarilla(data: DataFrame) -> DataFrame:
    df_ = data.shift()
    pivot = pd.Series(tp(df_), name="pivot")
    
    s1 =  df_['close']+(1.1*(df_['high']-df_['low'])/12)
    s2 = df_['close']-(1.1*(df_['high']-df_['low'])/6)
    s3 = df_['close']-(1.1*(df_['high']-df_['low'])/4)
    s4 =df_['close']-(1.1*(df_['high']-df_['low'])/2)
   
    

    r1 = df_['close']+(1.1*(df_['high']-df_['low'])/12)
    r2 = df_['close']+(1.1*(df_['high']-df_['low'])/6)
    r3 =df_['close']+(1.1*(df_['high']-df_['low'])/4)
    r4 = df_['close']+(1.1*(df_['high']-df_['low'])/2)
   
    return pd.concat(
            [
                pivot,
                pd.Series(s1, name="s1"),
                pd.Series(s2, name="s2"),
                pd.Series(s3, name="s3"),
                pd.Series(s4, name="s4"),
                pd.Series(r1, name="r1"),
                pd.Series(r2, name="r2"),
                pd.Series(r3, name="r3"),
                pd.Series(r4, name="r4"),
                            ],
            axis=1,
        )
pivot_camarilla(df)

In [7]:
def ppo(data: DataFrame,period_fast: int = 12,period_slow: int = 26,
    signal: int = 9,column: str = "close",
      adjust: bool = True,) -> DataFrame:

    EMA_fast = pd.Series(
        data[column].ewm(ignore_na=False, span=period_fast, adjust=adjust).mean(),
        name="EMA_fast",
    )
    EMA_slow = pd.Series(
        data[column].ewm(ignore_na=False, span=period_slow, adjust=adjust).mean(),
        name="EMA_slow",
    )
    PPO = pd.Series(((EMA_fast - EMA_slow) / EMA_slow) * 100, name="PPO")
    PPO_signal = pd.Series(
        PPO.ewm(ignore_na=False, span=signal, adjust=adjust).mean(), name="SIGNAL"
    )
    PPO_histo = pd.Series(PPO - PPO_signal, name="HISTO")

    return pd.concat([PPO, PPO_signal, PPO_histo], axis=1)

ppo(df)


,PPO,SIGNAL,HISTO
0,0.000000,0.000000,0.000000
1,0.006815,0.003786,0.003029
2,-0.097958,-0.037912,-0.060046
3,-0.193949,-0.090770,-0.103179
4,-0.323927,-0.160129,-0.163798
...,...,...,...
4625,4.543483,4.033759,0.509724
4626,4.370486,4.101104,0.269381
4627,4.010771,4.083038,-0.072266
4628,3.585299,3.983490,-0.398191


In [33]:
def vw_macd(data: DataFrame,period_fast: int = 12,period_slow: int = 26,
        signal: int = 9,column: str = "close",
        adjust: bool = True,
    ) -> DataFrame:
    
    vp = data["volume"] * data[column]
    _fast = pd.Series(
        (vp.ewm(ignore_na=False, span=period_fast, adjust=adjust).mean())
        / (
            data["volume"]
            .ewm(ignore_na=False, span=period_fast, adjust=adjust).mean()
            
        ),
        name="_fast",
    )

    _slow = pd.Series(
        (vp.ewm(ignore_na=False, span=period_slow, adjust=adjust).mean())
        / (
            data["volume"]
            .ewm(ignore_na=False, span=period_slow, adjust=adjust)
            .mean()
        ),
        name="_slow",
    )

    MACD = pd.Series(_fast - _slow, name="MACD")
    MACD_signal = pd.Series(
        MACD.ewm(ignore_na=False, span=signal, adjust=adjust).mean(), name="SIGNAL"
    )

    return pd.concat([MACD, MACD_signal], axis=1)
vw_macd(df)


,MACD,SIGNAL
0,0.000000,0.000000
1,0.008765,0.004870
2,-0.146014,-0.056968
3,-0.221255,-0.112621
4,-0.429676,-0.206937
...,...,...
4625,57.713421,51.338432
4626,58.418738,52.754493
4627,55.990015,53.401598
4628,52.195448,53.160368


In [30]:
def vw_macd(data: DataFrame,period_fast: int = 12,period_slow: int = 26,
        signal: int = 9,column: str = "close",
        adjust: bool = True,
    ) -> DataFrame:

    MACD = pd.Series(vwma(data,period=period_fast)-vwma(data,period=period_slow), 
                     name="VW MACD")
    print(MACD)
   
    MACD_signal = pd.Series(
        MACD.ewm(span=signal, adjust=adjust).mean(),
        name="MACD Signal"
    )

    return pd.concat([MACD, MACD_signal], axis=1)
vw_macd(df)


0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
           ...    
4625     76.974959
4626     80.873456
4627    102.436008
4628    104.275803
4629    105.220975
Name: MACD, Length: 4630, dtype: float64


,MACD,MACD Signal
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
4625,76.974959,64.622392
4626,80.873456,67.872605
4627,102.436008,74.785285
4628,104.275803,80.683389


In [6]:
def vwma(data: DataFrame,period: int = 20,column: str = "close",
        adjust: bool = True,
    ) -> DataFrame:
    
    cv=(data[column]*data['volume']).rolling(window=period,min_periods=1).sum()
    v=data['volume'].rolling(window=period,min_periods=1).sum()
    
    return pd.Series(cv/v,name='VWMA')
vwma(df)

0        164.350000
1        164.705481
2        161.640851
3        160.643955
4        158.507569
           ...     
4626    2289.757828
4627    2295.436658
4628    2297.989183
4629    2306.573349
4630    2310.582516
Name: VWMA, Length: 4631, dtype: float64

In [8]:
def ev_macd(data: DataFrame,period_fast: int = 20,period_slow: int = 40,
            signal: int = 9,adjust: bool = True,) -> DataFrame:
       
        evwma_slow = evwma(data, period_slow)

        evwma_fast = evwma(data, period_fast)

        MACD = pd.Series(evwma_fast - evwma_slow, name="EV MACD")
        MACD_signal = pd.Series(
            MACD.ewm(ignore_na=False, span=signal, adjust=adjust).mean(), name="SIGNAL"
        )

        return pd.concat([MACD, MACD_signal], axis=1)
ev_macd(df)

,EV MACD,SIGNAL
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
4626,100.116866,101.297997
4627,99.011518,100.840701
4628,97.881219,100.248805
4629,97.035431,99.606130


In [19]:
def mom(data: DataFrame, period: int = 10, column: str = "close") -> Series:

        return pd.Series(data[column].diff(period), 
                         name=f'{period}_MOM'
                        )
mom(df)

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
4626    101.95
4627      3.80
4628    -56.60
4629    -74.75
4630   -117.85
Name: 10_MOM, Length: 4631, dtype: float64

In [29]:
def roc(data: DataFrame, period: int = 12, column: str = "close") -> Series:
    return pd.Series(
        (data[column].diff(period) / data[column].shift(period)) * 100, 
        name="ROC"
    )
roc(df)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
4626    11.166876
4627     1.407627
4628     0.975876
4629    -1.736861
4630    -3.931707
Name: ROC, Length: 4631, dtype: float64

In [19]:
def tr(data: DataFrame) -> Series:
        TR1 = pd.Series(data["high"] - data["low"]).abs()
        TR2 = pd.Series(data["high"] - data["close"].shift()).abs()
        TR3 = pd.Series(data["close"].shift() - data["low"]).abs()
        _TR = pd.concat([TR1, TR2, TR3], axis=1)
        _TR["TR"] = _TR.max(axis=1)
        return pd.Series(_TR["TR"], 
                         name="TR"
                        )
tr(df)

0        3.00
1        3.20
2        8.05
3        3.75
4        6.20
        ...  
4626    38.50
4627    63.40
4628    62.55
4629    63.05
4630    79.40
Name: TR, Length: 4631, dtype: float64

In [20]:
def atr(data: DataFrame, period: int = 14) -> Series:
        TR = tr(data)
        return pd.Series(
            TR.rolling(center=False, window=period, 
                       min_periods=1).mean(),
            name=f'{period}  ATR'
        )
atr(df)

0        3.000000
1        3.100000
2        4.750000
3        4.500000
4        4.840000
          ...    
4626    66.007143
4627    68.892857
4628    68.875000
4629    59.653571
4630    60.403571
Name: 14  ATR, Length: 4631, dtype: float64

In [26]:
def vbm(data: DataFrame,roc_period: int = 22,atr_period: int = 65,
    column: str = "close") -> Series:

    return pd.Series(
        (
            (data[column].diff(roc_period) - data[column].shift(roc_period))
            / atr(data, atr_period)
        ),
        name="VBM"
    )
vbm(df)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
4626   -34.464984
4627   -36.907284
4628   -37.730390
4629   -37.391287
4630   -37.688160
Name: VBM, Length: 4631, dtype: float64

In [10]:
def rsi(data: DataFrame, period: int = 14,column: str = "close",
    adjust: bool = True,) -> Series:
    delta = data[column].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    _gain = up.ewm(alpha=1.0 / period, adjust=adjust).mean()
    _loss = down.abs().ewm(alpha=1.0 / period, adjust=adjust).mean()

    RS = _gain / _loss
    return pd.Series(100 - (100 / (1 + RS)), 
                     name=f'{period} period RSI'
                    )
rsi(df)

0              NaN
1       100.000000
2         7.656066
3         5.505245
4         3.822931
           ...    
4626     72.882572
4627     62.976062
4628     58.162448
4629     55.668065
4630     52.433991
Name: 14 period RSI, Length: 4631, dtype: float64

In [11]:
#not verified
def ift_rsi(data: DataFrame,column: str = "close",rsi_period: int = 5,
           wma_period: int = 9,) -> Series:
    v1 = pd.Series(0.1 * (rsi(data, rsi_period) - 50), name="v1")
    d = (wma_period * (wma_period + 1)) / 2 
    weights = np.arange(1, wma_period + 1)

    def linear(w):
        def _compute(x):
            return (w * x).sum() / d

        return _compute

    _wma = v1.rolling(wma_period, min_periods=wma_period)
    v2 = _wma.apply(linear(weights), raw=True)

    ift = pd.Series(
        ((v2 ** 2 - 1) / (v2 ** 2 + 1)), 
        name="IFT_RSI"
    )

    return ift
ift_rsi(df)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
4626    0.739059
4627    0.507591
4628   -0.126978
4629   -0.989858
4630   -0.364290
Name: IFT_RSI, Length: 4631, dtype: float64

In [12]:
def dmi(data: DataFrame, column: str = "close", adjust: bool = True
) -> Series:
    def _get_time(close):
        sd = close.rolling(5).std()
        asd = sd.rolling(10).mean()
        v = sd / asd
        t = 14 / v.round()
        t[t.isna()] = 0
        t = t.map(lambda x: int(min(max(x, 5), 30)))
        return t
    def _dmi(index):
        time = t.iloc[index]
        if (index - time) < 0:
            subset = data.iloc[0:index]
        else:
            subset = data.iloc[(index - time) : index]
        return rsi(subset, period=time, adjust=adjust).values[-1]
    dates = Series(data.index)
    periods = Series(range(14, len(dates)), index=dates.index[14:].values)
    t = _get_time(data[column])
    return periods.map(lambda x: _dmi(x))
dmi(df)

14      19.419438
15      28.416590
16      32.881534
17      36.887879
18      29.287396
          ...    
4626    76.771344
4627    73.698780
4628    30.470267
4629    15.952452
4630    40.108381
Length: 4617, dtype: float64

In [4]:
def sar(data: DataFrame, af: int = 0.02, amax: int = 0.2) -> Series:
        
        high, low = data.high, data.low

        # Starting values
        sig0, xpt0, af0 = True, high[0], af
        _sar = [low[0] - (high - low).std()]

        for i in range(1, len(data)):
            sig1, xpt1, af1 = sig0, xpt0, af0

            lmin = min(low[i - 1], low[i])
            lmax = max(high[i - 1], high[i])

            if sig1:
                sig0 = low[i] > _sar[-1]
                xpt0 = max(lmax, xpt1)
            else:
                sig0 = high[i] >= _sar[-1]
                xpt0 = min(lmin, xpt1)

            if sig0 == sig1:
                sari = _sar[-1] + (xpt1 - _sar[-1]) * af1
                af0 = min(amax, af1 + af)

                if sig0:
                    af0 = af0 if xpt0 > xpt1 else af1
                    sari = min(sari, lmin)
                else:
                    af0 = af0 if xpt0 < xpt1 else af1
                    sari = max(sari, lmax)
            else:
                af0 = af
                sari = xpt0

            _sar.append(sari)

        return pd.Series(_sar, index=data.index)
sar(df)

0        143.672158
1        144.138715
2        144.595940
3        145.044022
4        145.483141
           ...     
4626    2375.200000
4627    2454.950000
4628    2454.950000
4629    2448.912000
4630    2437.994280
Length: 4631, dtype: float64

In [5]:
def psar(data: DataFrame, iaf: int = 0.02, maxaf: int = 0.2) -> DataFrame:
        length = len(data)
        high, low, close = data.high, data.low, data.close
        psar = close[0 : len(close)]
        psarbull = [None] * length
        psarbear = [None] * length
        bull = True
        af = iaf
        hp = high[0]
        lp = low[0]

        for i in range(2, length):
            if bull:
                psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
            else:
                psar[i] = psar[i - 1] + af * (lp - psar[i - 1])

            reverse = False

            if bull:
                if low[i] < psar[i]:
                    bull = False
                    reverse = True
                    psar[i] = hp
                    lp = low[i]
                    af = iaf
            else:
                if high[i] > psar[i]:
                    bull = True
                    reverse = True
                    psar[i] = lp
                    hp = high[i]
                    af = iaf

            if not reverse:
                if bull:
                    if high[i] > hp:
                        hp = high[i]
                        af = min(af + iaf, maxaf)
                    if low[i - 1] < psar[i]:
                        psar[i] = low[i - 1]
                    if low[i - 2] < psar[i]:
                        psar[i] = low[i - 2]
                else:
                    if low[i] < lp:
                        lp = low[i]
                        af = min(af + iaf, maxaf)
                    if high[i - 1] > psar[i]:
                        psar[i] = high[i - 1]
                    if high[i - 2] > psar[i]:
                        psar[i] = high[i - 2]

            if bull:
                psarbull[i] = psar[i]
            else:
                psarbear[i] = psar[i]

        psar = pd.Series(psar, name="psar", index=data.index)
        psarbear = pd.Series(psarbull, name="psarbear", index=data.index)
        psarbull = pd.Series(psarbear, name="psarbull", index=data.index)

        return pd.concat([psar, psarbull, psarbear], axis=1)
psar(df)
    


,psar,psarbull,psarbear
0,164.350000,NaN,NaN
1,164.850000,NaN,NaN
2,167.000000,NaN,NaN
3,166.824000,NaN,NaN
4,166.411040,NaN,NaN
...,...,...,...
4626,2454.950000,NaN,NaN
4627,2454.950000,NaN,NaN
4628,2450.168000,NaN,NaN
4629,2441.397920,NaN,NaN


In [18]:
def bbands(
        data: DataFrame,
        period: int = 20,
        MA: Series = None,
        column: str = "close",
        std_multiplier: float = 2,
    ) -> DataFrame:
    

        std = data[column].rolling(window=period).std()

        if not isinstance(MA, pd.core.series.Series):
            middle_band = pd.Series(sma(data, period), name="BB_MIDDLE")
        else:
            middle_band = pd.Series(MA, name="BB_MIDDLE")

        upper_bb = pd.Series(middle_band + (std_multiplier * std), name="BB_UPPER")
        lower_bb = pd.Series(middle_band - (std_multiplier * std), name="BB_LOWER")

        return pd.concat([upper_bb, middle_band, lower_bb], axis=1)
bbands(df)

,BB_UPPER,BB_MIDDLE,BB_LOWER
0,NaN,164.350000,NaN
1,NaN,164.600000,NaN
2,NaN,162.816667,NaN
3,NaN,161.375000,NaN
4,NaN,159.870000,NaN
...,...,...,...
4626,2557.951988,2258.827500,1959.703012
4627,2555.157664,2274.420000,1993.682336
4628,2546.025472,2288.187500,2030.349528
4629,2536.386790,2299.420000,2062.453210


In [6]:
def vc(data: DataFrame, period: int = 5) -> DataFrame:
       
        float_axis = ((data.high + data.low) / 2).rolling(window=period).mean()
        vol_unit = (data.high - data.low).rolling(window=period).mean() * 0.2

        value_chart_high = pd.Series((data.high - float_axis) / vol_unit, name="Value Chart High")
        value_chart_low = pd.Series((data.low - float_axis) / vol_unit, name="Value Chart Low")
        value_chart_close = pd.Series((data.close - float_axis) / vol_unit, name="Value Chart Close")
        value_chart_open = pd.Series((data.open - float_axis) / vol_unit, name="Value Chart Open")

        return pd.concat([value_chart_high, value_chart_low, value_chart_close, value_chart_open], axis=1)
vc(df)

,Value Chart High,Value Chart Low,Value Chart Close,Value Chart Open
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,-2.778926,-9.183884,5.341983,-2.778926
...,...,...,...,...
4626,0.745290,-2.993300,4.750923,-0.580210
4627,-0.295749,-6.257758,4.984484,-1.866184
4628,-1.897711,-7.579942,5.698401,-3.582849
4629,-2.924080,-8.369753,6.697437,-3.442304


In [7]:
def williams_fractal(data: DataFrame, period: int = 2) -> DataFrame:
        def is_bullish_fractal(x):
            if x[period] == min(x):
                return True
            return False

        def is_bearish_fractal(x):
            if x[period] == max(x):
                return True
            return False

        window_size = period * 2 + 1
        bearish_fractals = pd.Series(
            data.high.rolling(window=window_size, center=True).apply(
                is_bearish_fractal, raw=True
            ),
            name="BearishFractal",
        )
        bullish_fractals = pd.Series(
            data.low.rolling(window=window_size, center=True).apply(
                is_bullish_fractal, raw=True
            ),
            name="BullishFractal",
        )
        return pd.concat([bearish_fractals, bullish_fractals], axis=1)
williams_fractal(df)

,BearishFractal,BullishFractal
0,NaN,NaN
1,NaN,NaN
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0
...,...,...
4626,0.0,0.0
4627,0.0,0.0
4628,0.0,0.0
4629,NaN,NaN


In [10]:
def evstc(data: DataFrame,period_fast: int = 12,period_slow: int = 30,
        k_period: int = 10,d_period: int = 3,adjust: bool = True) -> Series:
        
        ema_slow = evwma(data, period_slow)
        ema_fast = evwma(data, period_fast)

        macd = ema_fast - ema_slow

        STOK = pd.Series((
            (macd - macd.rolling(window=k_period).min())
            / (macd.rolling(window=k_period).max() - macd.rolling(window=k_period).min())
            ) * 100)

        STOD = STOK.rolling(window=d_period).mean()
        STOD_DoubleSmooth = STOD.rolling(window=d_period).mean()

        return pd.Series(STOD_DoubleSmooth, name="{0} period EVSTC".format(k_period))
evstc(df)

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
4626     77.375259
4627     92.810351
4628    100.000000
4629    100.000000
4630    100.000000
Name: 10 period EVSTC, Length: 4631, dtype: float64

In [11]:
def stc(data: DataFrame,period_fast: int = 23,period_slow: int = 50,k_period: int = 10,
        d_period: int = 3,column: str = "close",adjust: bool = True) -> Series:
        EMA_fast = pd.Series(
            data[column].ewm(ignore_na=False, span=period_fast, adjust=adjust).mean(),
            name="EMA_fast",
        )

        EMA_slow = pd.Series(
            data[column].ewm(ignore_na=False, span=period_slow, adjust=adjust).mean(),
            name="EMA_slow",
        )

        MACD = pd.Series((EMA_fast - EMA_slow), name="MACD")

        STOK = pd.Series((
            (MACD - MACD.rolling(window=k_period).min())
            / (MACD.rolling(window=k_period).max() - MACD.rolling(window=k_period).min())
            ) * 100)

        STOD = STOK.rolling(window=d_period).mean()
        STOD_DoubleSmooth = STOD.rolling(window=d_period).mean()  # "double smoothed"
        return pd.Series(STOD_DoubleSmooth, name="{0} period STC".format(k_period))
stc(df)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
4626    100.0
4627    100.0
4628    100.0
4629    100.0
4630    100.0
Name: 10 period STC, Length: 4631, dtype: float64

In [12]:
def msd(data: DataFrame, period: int = 21, column: str = "close") -> Series:
        return pd.Series(data[column].rolling(period).std(), name="MSD")
msd(df)

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
4626    156.464241
4627    169.310573
4628    174.853203
4629    176.730884
4630    174.977817
Name: MSD, Length: 4631, dtype: float64

In [6]:
def vfi(data: DataFrame,period: int = 130,smoothing_factor: int = 3,factor: int = 0.2,
        vfactor: int = 2.5,adjust: bool = True,) -> Series:
        typical = tp(data)
        inter = typical.apply(np.log).diff()
        vinter = inter.rolling(window=30).std()
        cutoff = pd.Series(factor * vinter * data["close"], name="cutoff")
        price_change = pd.Series(typical.diff(), name="pc")  
        mav = pd.Series(
            data["volume"].rolling(center=False, window=period).mean(), name="mav",
        )

        _va = pd.concat([data["volume"], mav.shift()], axis=1)
        _mp = pd.concat([price_change, cutoff], axis=1)
        _mp.fillna(value=0, inplace=True)

        def _vol_added(row):
            if row["volume"] > vfactor * row["mav"]:
                return vfactor * row["mav"]
            else:
                return row["volume"]

        added_vol = _va.apply(_vol_added, axis=1)

        def _multiplier(row):
            if row["pc"] > row["cutoff"]:
                return 1
            elif row["pc"] < 0 - row["cutoff"]:
                return -1
            else:
                return 0

        multiplier = _mp.apply(_multiplier, axis=1)
        raw_sum = (multiplier * added_vol).rolling(window=period).sum()
        raw_value = raw_sum / mav.shift()

        vfi = pd.Series(
            raw_value.ewm(
                ignore_na=False,
                min_periods=smoothing_factor - 1,
                span=smoothing_factor,
                adjust=adjust,
            ).mean(),
            name="VFI",
        )

        return vfi
vfi(df)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
4626    37.517837
4627    37.939932
4628    38.284983
4629    38.773712
4630    38.363603
Name: VFI, Length: 4631, dtype: float64

In [9]:
def fve(data: DataFrame, period: int = 22, factor: int = 0.3) -> Series:
        hl2 = (data["high"] + data["low"]) / 2
        tp_ = tp(data)
        smav = data["volume"].rolling(window=period).mean()
        mf = pd.Series((data["close"] - hl2 + tp_.diff()), name="mf")
        _mf = pd.concat([data["close"], data["volume"], mf], axis=1)

        def vol_shift(row):

            if row["mf"] > factor * row["close"] / 100:
                return row["volume"]
            elif row["mf"] < -factor * row["close"] / 100:
                return -row["volume"]
            else:
                return 0

        _mf["vol_shift"] = _mf.apply(vol_shift, axis=1)
        _sum = _mf["vol_shift"].rolling(window=period).sum()

        return pd.Series((_sum / smav) / period * 100)
fve(df)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
4626    77.886716
4627    72.295731
4628    68.193727
4629    66.741687
4630    64.157089
Length: 4631, dtype: float64

In [49]:
def vpt(data: DataFrame) -> Series:
        hilow = (data["high"] - data["low"]) * 100
        openclose = (data["close"] - data["open"]) * 100
        vol = data["volume"] / hilow
        spreadvol = (openclose * vol).cumsum()

        vpt = spreadvol + spreadvol

        return pd.Series(vpt, name="VPT")
vpt(df)

0      -2.781086e+05
1       1.340140e+04
2       4.033917e+05
3       2.388801e+06
4       5.354355e+06
            ...     
4626   -3.859559e+09
4627   -3.858372e+09
4628   -3.856903e+09
4629   -3.855705e+09
4630   -3.854386e+09
Name: VPT, Length: 4631, dtype: float64

In [24]:
def sqzmi(data: DataFrame, period: int = 20, MA: Series = None) -> DataFrame:

    if not isinstance(MA, pd.core.series.Series):
        ma = pd.Series(sma(data, period))
    else:
        ma = None

    bb = bbands(data, period=period, MA=ma)
    kc_ = kc(data, period=period, kc_mult=1.5)
    comb = pd.concat([bb, kc_], axis=1)

    def sqz_on(row):
        if row["BB_LOWER"] > row["KC_LOWER"] and row["BB_UPPER"] < row["KC_UPPER"]:
            return True
        else:
            return False

    comb["SQZ"] = comb.apply(sqz_on, axis=1)

    return pd.Series(comb["SQZ"], name="{0} period SQZMI".format(period))
sqzmi(df)

0       False
1       False
2       False
3       False
4       False
        ...  
4626    False
4627    False
4628    False
4629    False
4630    False
Name: 20 period SQZMI, Length: 4631, dtype: bool

In [30]:
def apz(data: DataFrame,period: int = 21,dev_factor: int = 2,
    MA: Series = None,adjust: bool = True,) -> DataFrame:
    if not isinstance(MA, pd.Series):
        MA = dema(data, period)
    price_range = pd.Series(
        (data["high"] - data["low"]).ewm(span=period, adjust=adjust).mean()
    )
    volatility_value = pd.Series(
        price_range.ewm(span=period, adjust=adjust).mean(), name="vol_val"
    )

    upper_band = pd.Series((volatility_value * dev_factor) + MA, name="UPPER")
    lower_band = pd.Series(MA - (volatility_value * dev_factor), name="LOWER")

    return pd.concat([upper_band, lower_band], axis=1)
apz(df)

,UPPER,LOWER
0,170.350000,158.350000
1,170.846372,158.626871
2,173.585801,158.649959
3,174.546990,158.641596
4,175.077422,158.220925
...,...,...
4626,2420.670874,2222.748589
4627,2455.042561,2255.245668
4628,2483.652100,2281.889388
4629,2506.539687,2302.735495


In [31]:
def ichimoku(data: DataFrame,tenkan_period: int = 9,kijun_period: int = 26,
    senkou_period: int = 52,chikou_period: int = 26,) -> DataFrame:
    tenkan_sen = pd.Series(
        (
            data["high"].rolling(window=tenkan_period).max()
            + data["low"].rolling(window=tenkan_period).min()
        )
        / 2,
        name="TENKAN",
    )  ## conversion line

    kijun_sen = pd.Series(
        (
            data["high"].rolling(window=kijun_period).max()
            + data["low"].rolling(window=kijun_period).min()
        )
        / 2,
        name="KIJUN",
    )  ## base line

    senkou_span_a = pd.Series(
        ((tenkan_sen + kijun_sen) / 2), name="senkou_span_a"
    ) .shift(kijun_period) ## Leading span

    senkou_span_b = pd.Series(
        (
            (
                data["high"].rolling(window=senkou_period).max()
                + data["low"].rolling(window=senkou_period).min()
            )
            / 2
        ),
        name="SENKOU",
    ).shift(kijun_period)

    chikou_span = pd.Series(
        data["close"].shift(-chikou_period),
        name="CHIKOU",
    )

    return pd.concat(
        [tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span], axis=1
    )
ichimoku(df)

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU
0,NaN,NaN,NaN,NaN,152.597078
1,NaN,NaN,NaN,NaN,152.055195
2,NaN,NaN,NaN,NaN,151.534987
3,NaN,NaN,NaN,NaN,151.035587
4,NaN,NaN,NaN,NaN,139.050000
...,...,...,...,...,...
4626,2381.525,2214.975,2024.5750,1930.125,NaN
4627,2392.600,2214.975,2025.5250,1930.125,NaN
4628,2379.475,2214.975,2026.1625,1930.125,NaN
4629,2360.950,2214.975,2023.4375,1930.125,NaN


In [32]:
def fish(data: DataFrame, period: int = 10, adjust: bool = True) -> Series:
    from numpy import log, seterr

    seterr(divide="ignore")

    med = (data["high"] + data["low"]) / 2
    ndaylow = med.rolling(window=period).min()
    ndayhigh = med.rolling(window=period).max()
    raw = (2 * ((med - ndaylow) / (ndayhigh - ndaylow))) - 1
    smooth = raw.ewm(span=5, adjust=adjust).mean()
    _smooth = smooth.fillna(0)

    return pd.Series(
        (log((1 + _smooth) / (1 - _smooth))).ewm(span=3, adjust=adjust).mean(),
        name="{0} period FISH.".format(period),
    )
fish(df)

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
4626    3.891030
4627    2.360945
4628    1.109067
4629    0.153473
4630   -0.596703
Name: 10 period FISH., Length: 4631, dtype: float64

In [43]:
def wto(data: DataFrame,channel_length: int = 10,average_length: int = 21,adjust: bool = True,) -> DataFrame:

    ap = tp(data)
    esa = ap.ewm(span=average_length, adjust=adjust).mean()
    d = pd.Series(
        (ap - esa).abs().ewm(span=channel_length, adjust=adjust).mean(), name="d"
    )
    ci = (ap - esa) / (0.015 * d)

    wt1 = pd.Series(ci.ewm(span=average_length, adjust=adjust).mean(), name="WT1.")
    wt2 = pd.Series(wt1.rolling(window=4).mean(), name="WT2.")

    return pd.concat([wt1, wt2], axis=1)
wto(df)

,WT1.,WT2.
0,NaN,NaN
1,-121.212121,NaN
2,-128.618255,NaN
3,-138.803618,NaN
4,-137.286233,-131.480057
...,...,...
4626,85.068211,87.567821
4627,82.463909,85.704906
4628,79.104644,83.366040
4629,74.949499,80.396566


In [6]:
def tp(data: DataFrame) -> Series:
        return pd.Series((data["high"] + data["low"] + data["close"]) / 3, name="TP")
tp(df)

0        165.116667
1        164.716667
2        161.233333
3        157.933333
4        154.916667
           ...     
4626    2393.700000
4627    2358.516667
4628    2327.716667
4629    2297.966667
4630    2284.183333
Name: TP, Length: 4631, dtype: float64

In [7]:
def kc(ohlc: DataFrame,period: int = 20,atr_period: int = 10,MA: Series = None,kc_mult: float = 2,) -> DataFrame:
        """Keltner Channels [KC]"""

        if not isinstance(MA, pd.core.series.Series):
            middle = pd.Series(ema(ohlc, period), name="KC_MIDDLE")
        else:
            middle = pd.Series(MA, name="KC_MIDDLE")

        up = pd.Series(middle + (kc_mult * atr(ohlc, atr_period)), name="KC_UPPER")
        down = pd.Series(
            middle - (kc_mult * atr(ohlc, atr_period)), name="KC_LOWER"
        )

        return pd.concat([up, down], axis=1)

In [13]:
def qstick( data: DataFrame, period: int = 14) -> Series:
    """
    QStick indicator shows the dominance of black (down) or white (up) candlesticks, which are red and green in Chart,
    as represented by the average open to close change for each of past N days."""

    _close = data["close"].tail(period)
    _open = data["open"].tail(period)

    return pd.Series(
        (_close - _open) / period, name="{0} period QSTICK.".format(period))
qstick(df)

4617    2.957143
4618    2.278571
4619    0.050000
4620    1.778571
4621   -2.571429
4622    2.078571
4623    2.071429
4624    0.935714
4625   -2.457143
4626   -0.560714
4627   -2.910714
4628   -2.528571
4629   -1.932143
4630   -1.496429
Name: 14 period QSTICK., dtype: float64

In [21]:
def chandelier(data: DataFrame,short_period: int = 22,long_period: int = 22,
    k: int = 3,) -> DataFrame:
    """
    Chandelier Exit sets a trailing stop-loss based on the Average True Range (ATR).
    The indicator is designed to keep traders in a trend and prevent an early exit as long as the trend extends.
    Typically, the Chandelier Exit will be above prices during a downtrend and below prices during an uptrend.
    """

    l = pd.Series(
        data["high"].rolling(window=long_period).max() - atr(data, 22) * k,
        name="Long.",
    )
    s = pd.Series(
        data["low"].rolling(window=short_period).min() + atr(data, 22) * k,
        name="Short.",
    )

    return pd.concat([s, l], axis=1)
chandelier(df)

,Short.,Long.
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
4626,2152.700000,2285.300000
4627,2180.859091,2286.990909
4628,2189.286364,2282.763636
4629,2193.063636,2278.986364


In [22]:
def cmo(data: DataFrame,period: int = 9,factor: int = 100,column: str = "close",
    adjust: bool = True,) -> DataFrame:
    """
    Chande Momentum Oscillator (CMO) - technical momentum indicator invented by the technical analyst Tushar Chande.
    It is created by calculating the difference between the sum of all recent gains and the sum of all recent losses and then
    dividing the result by the sum of all price movement over the period.
    This oscillator is similar to other momentum indicators such as the Relative Strength Index and the Stochastic Oscillator
    because it is range bounded (+100 and -100)."""

    # get the price diff
    delta = data[column].diff()

    # positive gains (up) and negative gains (down) Series
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    # EMAs of ups and downs
    _gain = up.ewm(com=period, adjust=adjust).mean()
    _loss = down.ewm(com=period, adjust=adjust).mean().abs()

    return pd.Series(factor * ((_gain - _loss) / (_gain + _loss)), name="CMO")
cmo(df)

0              NaN
1       100.000000
2       -85.123967
3       -89.404840
4       -92.768216
           ...    
4626     43.173791
4627     17.094689
4628      5.071531
4629     -1.072220
4630     -8.970292
Name: CMO, Length: 4631, dtype: float64

In [25]:
def baspn( data: DataFrame, period: int = 40, adjust: bool = True) -> DataFrame:
    """
    Normalized BASP indicator
    """

    sp = data["high"] - data["close"]
    bp = data["close"] - data["low"]
    spavg = sp.ewm(span=period, adjust=adjust).mean()
    bpavg = bp.ewm(span=period, adjust=adjust).mean()

    nbp = bp / bpavg
    nsp = sp / spavg

    varg = data["volume"].ewm(span=period, adjust=adjust).mean()
    nv = data["volume"] / varg

    nbf = pd.Series((nbp * nv).ewm(span=20, adjust=adjust).mean(), name="Buy.")
    nsf = pd.Series((nsp * nv).ewm(span=20, adjust=adjust).mean(), name="Sell.")

    return pd.concat([nbf, nsf], axis=1)
baspn(df)

,Buy.,Sell.
0,1.000000,1.000000
1,1.691413,0.989690
2,1.660220,1.754750
3,1.326358,1.467724
4,1.531182,1.566548
...,...,...
4626,2.173136,1.349398
4627,1.979629,1.370603
4628,1.808987,1.357820
4629,1.682360,1.280817


In [26]:
def basp( data: DataFrame, period: int = 40, adjust: bool = True) -> DataFrame:
    """BASP indicator serves to identify buying and selling pressure."""

    sp = data["high"] - data["close"]
    bp = data["close"] - data["low"]
    spavg = sp.ewm(span=period, adjust=adjust).mean()
    bpavg = bp.ewm(span=period, adjust=adjust).mean()

    nbp = bp / bpavg
    nsp = sp / spavg

    varg = data["volume"].ewm(span=period, adjust=adjust).mean()
    nv = data["volume"] / varg

    nbfraw = pd.Series(nbp * nv, name="Buy.")
    nsfraw = pd.Series(nsp * nv, name="Sell.")

    return pd.concat([nbfraw, nsfraw], axis=1)
basp(df)

,Buy.,Sell.
0,1.000000,1.000000
1,2.316977,0.980361
2,1.606462,3.073221
3,0.503727,0.760495
4,2.173116,1.876272
...,...,...
4626,0.294585,0.481352
4627,0.141305,1.572052
4628,0.187893,1.236375
4629,0.479398,0.549296


In [31]:
def copp( data: DataFrame, adjust: bool = True) -> Series:
    """The Coppock Curve is a momentum indicator, it signals buying opportunities 
        when the indicator moved from negative territory to positive territory.
    """

    roc1 = roc(data, 14)
    roc2 = roc(data, 11)

    return pd.Series(
        (roc1 + roc2).ewm(span=10, min_periods=9, adjust=adjust).mean(),
        name="Coppock Curve",
    )
copp(df)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
4626    26.025556
4627    23.509507
4628    20.398772
4629    16.041729
4630    12.279053
Name: Coppock Curve, Length: 4631, dtype: float64

In [39]:
def cci( data: DataFrame, period: int = 20, constant: float = 0.015) -> Series:
    """Commodity Channel Index (CCI) is a versatile indicator that can be used to identify a new trend or warn of extreme conditions.
    CCI measures the current price level relative to an average price level over a given period of time.
    The CCI typically oscillates above and below a zero line. Normal oscillations will occur within the range of +100 and −100.
    Readings above +100 imply an overbought condition, while readings below −100 imply an oversold condition.
    As with other overbought/oversold indicators, this means that there is a large probability that the price will correct to more representative levels.
    source: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:commodity_channel_index_cci
    :param pd.DataFrame data: 'open, high, low, close' pandas DataFrame
    :period: int - number of periods to take into consideration
    :factor float: the constant at .015 to ensure that approximately 70 to 80 percent of CCI values would fall between -100 and +100.
    :return pd.Series: result is pandas.Series
    """

    TP = tp(data)
    TP_rolling = TP.rolling(window=period, min_periods=0)
    # calculate MAD (Mean Deviation)
    # https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/other-measures-of-spread/a/mean-absolute-deviation-mad-review
    mad = TP_rolling.apply(lambda s: abs(s - s.mean()).mean(), raw=True)
    return pd.Series(
        (TP - TP_rolling.mean()) / (constant * mad),
        name="{0} period CCI".format(period),
    )
cci(df)

0              NaN
1       -66.666667
2      -100.000000
3      -107.916667
4      -112.173359
           ...    
4626     67.677806
4627     46.260472
4628     25.087973
4629      0.491893
4630    -18.344355
Name: 20 period CCI, Length: 4631, dtype: float64